In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
vcfs=['vcf1', 'vcf2', 'vcf3']
beds=['bed1', 'bed2', 'bed3']
def print_args(**kargs):
    for k in kargs.keys():
        print(k)

print_args(vcfs=list1, tsvs=list2)

In [37]:
def import_VCF_to_pandas(vcf_file):
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()

    if first_line.startswith('##'):
        df = pd.read_csv(vcf_file, sep='\t', skiprows=[header_lines], header=header_lines)
        
        df['ALT']=df['ALT'].str.upper()
        df['REF']=df['REF'].str.upper()
        #Check INFO
        if 'INFO' in df.columns:
            return df
        else:
            last_column = df.columns[-1]
            df = df.rename(columns={last_column: 'INFO'})
            return df
    else:
        print("This vcf file is not properly formatted")
        sys.exit(1)

In [121]:
vcf_info = "/home/laura/TMP/17488.combined.hf.SNP.final.vcf"
vcf_noinfo = "/home/laura/TMP/17488column.vcf"
tsv = "/home/laura/TMP/17488.tsv"
bed = "/home/laura/COVID/primers/nCoV-2019.bed"
bed2 = "/home/laura/COVID/primers/nCoV-2019_2.bed"

In [141]:
def annotate_vcfs(tsv_df, vcfs):
    df = pd.read_csv(tsv_df, sep="\t")
    for vcf in vcfs:
        print("ANNOTATING VCF: ", vcf)
        header = (".").join(vcf.split("/")[-1].split(".")[0:-1])
        dfvcf = import_VCF_to_pandas(vcf)
        dfvcf = dfvcf[['POS', 'REF', 'ALT', 'INFO']]
        dfvcf = dfvcf.rename(columns={'INFO': header})
        df = df.merge(dfvcf, how='left')
    return df



In [142]:
annotate_vcfs(tsv, [vcf_info, vcf_noinfo]).head()

ANNOTATING VCF:  /home/laura/TMP/17488.combined.hf.SNP.final.vcf
ANNOTATING VCF:  /home/laura/TMP/17488column.vcf


,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,ALT_FREQ,TOTAL_DP,PVAL,PASS,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,17488.combined.hf.SNP.final,17488column
0,NC_045512.2,2532,C,T,0,0,0,765,38,72,1.000000,765,0.0,True,NaN,NaN,NaN,NaN,NaN,AC=2;AF=1.00;AN=2;BaseQRankSum=1.80;DP=31;ExcessHet=0.0000;FS=0.000;InbreedingCoeff=0.9918;MQ=60.00;MQRankSum=0.00;QD=34.88;ReadPosRankSum=0.158;SOR=0.031,tets
1,NC_045512.2,3037,C,T,0,0,0,664,41,72,1.000000,664,0.0,True,cds-YP_009724389.1,TTC,F,TTT,F,NaN,NaN
2,NC_045512.2,4319,G,A,72,65,40,795,188,65,0.916955,867,0.0,True,cds-YP_009724389.1,GCC,A,ACC,T,NaN,NaN
3,NC_045512.2,4925,G,T,86,25,62,608,140,65,0.876081,694,0.0,True,cds-YP_009724389.1,GAC,D,TAC,Y,NaN,NaN
4,NC_045512.2,23174,A,C,0,0,0,949,171,67,1.000000,949,0.0,True,cds-YP_009724389.1,TAC,Y,TAT,Y,AC=2;AF=1.00;AN=2;BaseQRankSum=2.41;DP=80;ExcessHet=0.0000;FS=0.000;InbreedingCoeff=0.9996;MQ=59.98;MQRankSum=0.00;QD=34.29;ReadPosRankSum=1.73;SOR=0.076,"1/1:0,80:80:99:2790,240,0"


In [107]:
def bed_to_df(bed_file):
    """
    Import bed file separated by tabs into a pandas df
    -Handle header line
    -Handle with and without description (If there is no description adds true or false to annotated df)
    """
    header_lines = 0
    #Handle likely header by checking colums 2 and 3 as numbers
    with open(bed_file, 'r') as f:
        next_line = f.readline().strip()
        line_split = next_line.split(None) #This split by any blank character
        start = line_split[1]
        end = line_split[2]
        while not start.isdigit() and not end.isdigit():
            header_lines = header_lines + 1
            next_line = f.readline().strip()
            line_split = next_line.split(None) #This split by any blank character
            start = line_split[1]
            end = line_split[2]

    if header_lines == 0:
        df = pd.read_csv(bed_file, sep="\t", header=None) #delim_whitespace=True
    else:
        df = pd.read_csv(bed_file, sep="\t", skiprows=header_lines, header=None) #delim_whitespace=True

    df = df.iloc[:,0:4]
    df.columns = ["#CHROM", "start", "end", "description"]
        
    return df

In [114]:
bed_to_df(bed).head()

,#CHROM,start,end,description
0,MN908947.3,30,54,nCoV-2019_1_LEFT
1,MN908947.3,385,410,nCoV-2019_1_RIGHT
2,MN908947.3,320,342,nCoV-2019_2_LEFT
3,MN908947.3,704,726,nCoV-2019_2_RIGHT
4,MN908947.3,642,664,nCoV-2019_3_LEFT


In [117]:
def add_bed_info(bed_df, position):
    """
    Identify a position within a range
    credits: https://stackoverflow.com/questions/6053974/python-efficiently-check-if-integer-is-within-many-ranges
    """
    #dict_position = bed_to_dict(bed_file)
    if any(start <= position <= end for (start, end) in zip(bed_df.start.values.tolist(), bed_df.end.values.tolist())):
        description_out = bed_df.description[(bed_df.start <= position) & (bed_df.end >= position)].values[0]
        return description_out
    else:
        return None

In [134]:
def annotate_bed_s(tsv_df, bed_files):
    df = pd.read_csv(tsv_df, sep="\t")

    variable_list = [ x.split("/")[-1].split(".")[0] for x in bed_files] #extract file name and use it as header
    
    for variable_name, bed_file in zip(variable_list,bed_files):
        print("ANNOTATING BED: ", bed_file)
        bed_annot_df = bed_to_df(bed_file)
        df[variable_name] = df['POS'].apply(lambda x: add_bed_info(bed_annot_df,x))
    return df

In [136]:
annotate_bed_s(tsv, [bed, bed2).head()

ANNOTATING BED:  /home/laura/COVID/primers/nCoV-2019.bed
ANNOTATING BED:  /home/laura/COVID/primers/nCoV-2019_2.bed


,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,ALT_FREQ,TOTAL_DP,PVAL,PASS,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,nCoV-2019,nCoV-2019_2
0,NC_045512.2,2532,C,T,0,0,0,765,38,72,1.000000,765,0.0,True,NaN,NaN,NaN,NaN,NaN,None,None
1,NC_045512.2,3037,C,T,0,0,0,664,41,72,1.000000,664,0.0,True,cds-YP_009724389.1,TTC,F,TTT,F,None,None
2,NC_045512.2,4319,G,A,72,65,40,795,188,65,0.916955,867,0.0,True,cds-YP_009724389.1,GCC,A,ACC,T,nCoV-2019_15_LEFT,nCoV-2019_15_LEFT
3,NC_045512.2,4925,G,T,86,25,62,608,140,65,0.876081,694,0.0,True,cds-YP_009724389.1,GAC,D,TAC,Y,None,None
4,NC_045512.2,23174,A,C,0,0,0,949,171,67,1.000000,949,0.0,True,cds-YP_009724389.1,TAC,Y,TAT,Y,None,None


In [146]:
def user_annotation(tsv_file, **kargs):
    vcf_files = kargs['vcfs']
    bed_files = kargs['beds']

    bed_df = annotate_bed_s(tsv_file, bed_files)
    vcf_df = annotate_vcfs(tsv_file, vcf_files)

    df = bed_df.merge(vcf_df)

    return df


In [147]:
bedFiles = [bed, bed2]
vcfFiles = [vcf_info, vcf_noinfo]

user_annotation(tsv, vcfs=vcfFiles, beds=bedFiles)

ANNOTATING BED:  /home/laura/COVID/primers/nCoV-2019.bed
ANNOTATING BED:  /home/laura/COVID/primers/nCoV-2019_2.bed
ANNOTATING VCF:  /home/laura/TMP/17488.combined.hf.SNP.final.vcf
ANNOTATING VCF:  /home/laura/TMP/17488column.vcf


,REGION,POS,REF,ALT,REF_DP,REF_RV,REF_QUAL,ALT_DP,ALT_RV,ALT_QUAL,ALT_FREQ,TOTAL_DP,PVAL,PASS,GFF_FEATURE,REF_CODON,REF_AA,ALT_CODON,ALT_AA,nCoV-2019,nCoV-2019_2,17488.combined.hf.SNP.final,17488column
0,NC_045512.2,2532,C,T,0,0,0,765,38,72,1.000000,765,0.000000e+00,True,NaN,NaN,NaN,NaN,NaN,None,None,AC=2;AF=1.00;AN=2;BaseQRankSum=1.80;DP=31;ExcessHet=0.0000;FS=0.000;InbreedingCoeff=0.9918;MQ=60.00;MQRankSum=0.00;QD=34.88;ReadPosRankSum=0.158;SOR=0.031,tets
1,NC_045512.2,3037,C,T,0,0,0,664,41,72,1.000000,664,0.000000e+00,True,cds-YP_009724389.1,TTC,F,TTT,F,None,None,NaN,NaN
2,NC_045512.2,4319,G,A,72,65,40,795,188,65,0.916955,867,0.000000e+00,True,cds-YP_009724389.1,GCC,A,ACC,T,nCoV-2019_15_LEFT,nCoV-2019_15_LEFT,NaN,NaN
3,NC_045512.2,4925,G,T,86,25,62,608,140,65,0.876081,694,0.000000e+00,True,cds-YP_009724389.1,GAC,D,TAC,Y,None,None,NaN,NaN
4,NC_045512.2,23174,A,C,0,0,0,949,171,67,1.000000,949,0.000000e+00,True,cds-YP_009724389.1,TAC,Y,TAT,Y,None,None,AC=2;AF=1.00;AN=2;BaseQRankSum=2.41;DP=80;ExcessHet=0.0000;FS=0.000;InbreedingCoeff=0.9996;MQ=59.98;MQRankSum=0.00;QD=34.29;ReadPosRankSum=1.73;SOR=0.076,"1/1:0,80:80:99:2790,240,0"
5,NC_045512.2,27463,C,G,0,0,0,799,26,74,1.000000,799,0.000000e+00,True,cds-YP_009724389.1,CCT,P,CTT,L,nCoV-2019_91_LEFT,nCoV-2019_91_LEFT,AC=2;AF=1.00;AN=2;DP=74;ExcessHet=-0.0000;FS=0.000;InbreedingCoeff=1.0000;MQ=60.00;QD=30.96;SOR=1.212,"1|1:0,74:74:99:1|1:27463_C_G:3401,232,0:27463"
6,NC_045512.2,27469,G,A,0,0,0,673,191,63,0.998516,674,0.000000e+00,True,cds-YP_009724389.1,CCT,P,TCT,S,nCoV-2019_91_LEFT,nCoV-2019_91_LEFT,AC=2;AF=1.00;AN=2;BaseQRankSum=0.680;DP=79;ExcessHet=0.0000;FS=0.000;InbreedingCoeff=1.0000;MQ=60.00;MQRankSum=0.00;QD=29.87;ReadPosRankSum=0.819;SOR=1.321,"1|1:0,79:79:99:1|1:27463_C_G:3515,241,0:27463"
7,NC_045512.2,33457,C,T,0,0,0,14,1,64,0.933333,15,6.629320e-11,True,cds-YP_009724390.1,GAA,E,CAA,Q,None,None,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.759e+00;DP=82;ExcessHet=-0.0000;FS=0.000;InbreedingCoeff=1.0000;MQ=60.00;MQRankSum=0.00;QD=34.95;ReadPosRankSum=-4.430e-01;SOR=0.368,"1/1:0,82:82:99:2657,246,0"
8,NC_045512.2,33551,T,G,0,0,0,791,279,64,1.000000,791,0.000000e+00,True,cds-YP_009724390.1,AAC,N,AAT,N,None,None,AC=2;AF=1.00;AN=2;BaseQRankSum=1.71;DP=80;ExcessHet=-0.0000;FS=0.000;InbreedingCoeff=1.0000;MQ=60.00;MQRankSum=0.00;QD=33.81;ReadPosRankSum=-1.028e+00;SOR=0.376,"1/1:0,80:80:99:2855,239,0"
9,NC_045512.2,36538,C,T,0,0,0,1040,189,67,1.000000,1040,0.000000e+00,True,cds-YP_009724390.1,GAT,D,GGT,G,None,None,AC=2;AF=1.00;AN=2;BaseQRankSum=-1.730e+00;DP=99;ExcessHet=0.0000;FS=0.000;InbreedingCoeff=0.9798;MQ=60.00;MQRankSum=0.00;QD=34.55;ReadPosRankSum=0.668;SOR=0.518,testRandom
